## First Part-Navigation Message

##Mission of this study is to calculate the position of the GPS satellite using the navigation message.
I computed the PRN number of associated satellite by adding up the last three number of my school ID and I found it as 17.I computed epoch as 47040.
This codes are written by F.Burcu Arslan.

In [1]:
from numpy import *
import math as m

In [2]:
"""
Rotation about x axis
"""
def R1(α):
    return matrix([
            [1  ,  0  ,  0  ],
            [0,m.cos(α),m.sin(α)],
            [0,-m.sin(α),m.cos(α)]]
         )

"""
Rotation about y axis
"""
def R2(α):
    return matrix([
            [m.cos(α),0, -m.sin(α)],
            [0 ,  1   ,  0  ],
            [m.sin(α),0, m.cos(α)]]
      )

"""
Rotation about z axis
"""
def R3(α):
    return matrix([
            [m.cos(α),m.sin(α),0],
            [-m.sin(α),m.cos(α),0],
            [  0 ,  0  ,  1  ]]
      )


In [ ]:
c = 299792458 # in 𝑚/𝑠
µ = 3.986005*(10**14) # in m**3/s**2
wE = 7.2921151467*(10**(-5)) # in rad/sec


def cal_brd(eph,brd):
    eph = 47040 # in seconds of day,it's calculated using my school ID
    # brd = array(brd).reshape(7,4) #adjustments for the vector entered to be in 3*1 format
    
    # Crc, Crs orbital radius correction ,in meter
    # Cuc, Cus latitude argument correction, in radians
    # Cic, Cis inclination correction, in radians
    Crs = brd[0][1]   
    Δn = brd[0][2]   
    M0 = brd[0][3] # M0 in radians  
    Cuc = brd[1][0]  
    e = brd[1][1]  
    Cus = brd[1][2]  
    sqrta = brd[1][3]
    t0e = brd[2][0]  
    Cic = brd[2][1]  
    Ω0 = brd[2][2]  
    Cis = brd[2][3] # Ω0 in radians
    i0 = brd[3][0]  
    Crc = brd[3][1]  
    w = brd[3][2] # w is argument of perigee,omega,in radians  
    Ω1 = brd[3][3]  # Ω1 is Omega Dot, rad/sec
    i1 = brd[4][0] # i1 is IDOT(rate of inclination angle) rad/sec
    
    a = sqrta**2 # I pulled this value from the given ones for later use.
    
    t0e = t0e - (3*86400) # updated t0e,in seconds of day
    
    tk = eph - t0e
    Mk = M0 + ((m.sqrt(µ/a**3) + Δn) * tk)
    E0 = Mk 
    Ek = Mk + e*m.sin(E0)
    while abs(Ek - E0) < 1E-15:
        E0 = Ek
        Ek = Mk + e*m.sin(Ek)
        
    vk = m.atan((m.sqrt(1-e**2)* m.sin(Ek)) / (m.cos(Ek) - e)) # vk is the true anomaly
    uk = w + vk + (Cuc*m.cos(2*(w+vk))) + (Cus*m.sin(2*(w+vk))) # argument of latitude
    rk = a*(1 - e*m.cos(Ek)) + Crc*m.cos(2*(w+vk)) + Crs*m.sin(2*(w+vk)) # rk is the radial distance
    ik = i0 + i1*tk + Cic*m.cos(2*(w+vk)) + Cis*m.sin(2*(w+vk)) # inclination
    λk = Ω0 + (Ω1 - wE)*tk - wE*t0e # longitude of the ascending node
    
    pos = R3(-λk).dot(R1(-ik)).dot(R3(-uk)).dot(array([rk,0,0]).reshape(3,1))
    
    return pos  

In [8]:
cal_brd(47040,[[ 0.360000000000*(10**2), 0.879062500000*(10**2), 0.392016329054*(10**(-8)),-0.214407259609*10],
                [0.466033816338*(10**(-5)), 0.136141142575*(10**(-1)), 0.648945569992*(10**(-5)), 0.515372900581*(10**4)],
                [0.302400000000*(10**6), 0.234693288803*(10**(-6)), 0.289410748276*10, 0.180676579475*(10**(-6))],
                [0.975631445640*(10**0), 0.263031250000*(10**3),-0.141768597766*10,-0.769032033271*(10**(-8))],
                [-0.367872466222*(10**(-10)), 0.100000000000*10, 0.225100000000*(10**4), 0.000000000000],
                [0.200000000000*10, 0.000000000000,-0.111758708954*(10**(-7)), 0.360000000000*(10**2)],
                [0.295987000000*(10**6), 0.400000000000*10, 0.000000000000, 0.000000000000]],
    )

matrix([[ 23553590.98016134],
        [-12046512.97720966],
        [  2474316.74978707]])